Тренировочных данных очень мало, поэтому хороший результат можно получить, загрузив предобученную сеть, в данном случае была использована ResNet50.

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from keras.preprocessing import image
import os

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

num_classes = 2
resnet_weights_path = 'resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
model.add(Dense(num_classes, activation='softmax'))

model.layers[0].trainable = False

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

Попробуем модель с аугментацией:

добавим поворот, отражение, приближение и сдвиг по высоте и ширине. 

In [14]:
image_height = 256
image_width = 341
target_size = (image_width, image_height)


data_generator_with_aug = ImageDataGenerator(
        rotation_range=10, 
        zoom_range = 0.1, 
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,  
        vertical_flip=True)

train_generator = data_generator_with_aug.flow_from_directory(
        'train',
        target_size=target_size,
        batch_size=24,
        class_mode='categorical')


model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        epochs=36)

Found 40 images belonging to 2 classes.
Epoch 1/36
2/2 [==============================] - 11s 5s/step - loss: 0.0348 - acc: 1.0000
Epoch 2/36
2/2 [==============================] - 10s 5s/step - loss: 0.0419 - acc: 1.0000
Epoch 3/36
2/2 [==============================] - 10s 5s/step - loss: 0.0322 - acc: 1.0000
Epoch 4/36
2/2 [==============================] - 10s 5s/step - loss: 0.0264 - acc: 1.0000
Epoch 5/36
2/2 [==============================] - 10s 5s/step - loss: 0.0354 - acc: 1.0000
Epoch 6/36
2/2 [==============================] - 10s 5s/step - loss: 0.0325 - acc: 1.0000
Epoch 7/36
2/2 [==============================] - 10s 5s/step - loss: 0.0318 - acc: 1.0000
Epoch 8/36
2/2 [==============================] - 10s 5s/step - loss: 0.0273 - acc: 1.0000
Epoch 9/36
2/2 [==============================] - 10s 5s/step - loss: 0.0222 - acc: 1.0000
Epoch 10/36
2/2 [==============================] - 10s 5s/step - loss: 0.0277 - acc: 1.0000
Epoch 11/36
2/2 [==============================] 

В другой версии программы были взяты несколько фотографий для валидации, чтобы более объективно оценивать результат модели. (Результат одинаков)

In [5]:
"""
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = data_generator_no_aug.flow_from_directory(
        'val',
        target_size=target_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        epochs=36,
        validation_data=validation_generator,
        validation_steps=1)
"""

"\ndata_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)\nvalidation_generator = data_generator_no_aug.flow_from_directory(\n        'val',\n        target_size=target_size,\n        class_mode='categorical')\n\nmodel.fit_generator(\n        train_generator,\n        steps_per_epoch=3,\n        epochs=36,\n        validation_data=validation_generator,\n        validation_steps=1)\n"

Загрузим изображения и преобразуем в массив

In [15]:
_, _, test_filenames =next(os.walk('test'))
test_filenames = sorted(test_filenames)[1:]

In [16]:
img = image.load_img('test/' + test_filenames[0], target_size=target_size)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

for name in test_filenames[1:]:
    img = image.load_img('test/' + name, target_size=target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([images, x])
    
images.shape

(660, 341, 256, 3)

Построим предсказания и оценим их объективность:

In [17]:
classes = model.predict_classes(images, batch_size=30)
print("доля грязных тарелок = ", sum(classes == 1) / len(classes))

доля грязных тарелок =  0.6848484848484848


In [18]:
cleaned = [i for i in range(len(classes)) if classes[i] == 0]
#print(cleaned)

Для проверки вручную размечено 34 фотографии из теста для предварительной оценки точности. Проверим точность на них:

In [19]:
from sklearn.metrics import f1_score

really_cleaned = np.array([5,7,8,13,16,18,19,21,22,23,33])
real_dirty = [0 if i in really_cleaned else 1 for i in range(34)]
real_clean = [1 if i in really_cleaned else 0 for i in range(34)]


print("угадано грязных ", f1_score(real_dirty, classes[:34]))
print("угадано чистых ", f1_score(real_clean, 1 - classes[:34]))

угадано грязных  0.9787234042553191
угадано чистых  0.9523809523809523


Видно, что точность довольно высокая, модель адекватна. На кеггле получен результат 0.96969 (ник 01)

Преобразуем результаты в таблицу:

In [20]:
ind = list(map(lambda x: x[:-4], test_filenames))
pred = ['cleaned' if i == 0 else 'dirty' for i in classes]

data = pd.DataFrame()
data['id'] = ind

data['pred'] = pred
data.columns = ['id', 'label']
data.head()

,id,label
0,0000,dirty
1,0001,dirty
2,0002,dirty
3,0003,dirty
4,0004,dirty


In [21]:
data.to_csv('pred.csv', index=False)

Возможно, результат можно улучшить, подобрав дополнительные параметры для аугментации. Из опробованных текущие показали себя наилучшим образом.